# Raccolta features per SHOTS

In [1]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

cu = db.users.find()
cs = db.shots.find()
cf = db.followers.find()
cl = db.likes.find()
ct = db.teams.find()
cutfs = db.user_team_first_shot.find()
csk = db.skills.find()

In [2]:
usernames_no_repeated = []
for user in cutfs:
    if user['user'] not in usernames_no_repeated:
        usernames_no_repeated.append(user['user'])

In [3]:
len(usernames_no_repeated)

13282

### Tags più frequenti

In [4]:
%%time

tags_found = []

i=0

for u in usernames_no_repeated:
    active_user = db.shots.find({'username': u})
    if active_user.count() > 0:
        for s in active_user[0]['shots']:
            tags = s['tags']
            for t in tags:
                found = False
                for elem in tags_found:
                    if elem['tag'] == t:
                        elem.update({'count': elem['count']+1})
                        found = True
                        break

                if not found:
                    tf = {'tag': t, 'count': 1}
                    tags_found.append(tf)

                #print t   
            
    i=i+1
    print i
        
for t in tags_found:
    print t

/home/simo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


KeyboardInterrupt: 

In [ ]:
len(tags_found)

scrivere su file

In [ ]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/shots_tags_frequences.txt", mode='a') as f:
    for t in tags_found:
        f.write(str(t['tag'])+' - '+str(t['count'])+'\n')

leggere da file

In [5]:
tags_found = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/shots_tags_frequences.txt", "r") as ins:
    for line in ins:
        tag = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'tag': tag, 'count': count}
        tags_found.append(elem)

Stampa degli n tag piu frequenti


In [6]:
def n_most_frequent(n, tag_list):
    max_tag_prev = {'count':1000000}
    for i in range(0,n):
        max = 0
        max_tag = {}
        for t in tag_list:
            if int(t['count']) > max and int(t['count']) < int(max_tag_prev['count']):
                max = int(t['count'])
                max_tag = t
                
        
        max_tag_prev = max_tag
        print max_tag

In [7]:
len(tags_found)

128350

In [8]:
n_most_frequent(10, tags_found)

{'count': '87433', 'tag': 'illustration'}
{'count': '80551', 'tag': 'ui'}
{'count': '56308', 'tag': 'design'}
{'count': '54899', 'tag': 'logo'}
{'count': '43086', 'tag': 'ux'}
{'count': '41230', 'tag': 'app'}
{'count': '38965', 'tag': 'icon'}
{'count': '31255', 'tag': 'typography'}
{'count': '30267', 'tag': 'web'}
{'count': '28828', 'tag': 'animation'}


### Shtos con più tags

In [ ]:
shots_tags_count = []

i=0

for u in usernames_no_repeated:
    active_user = db.shots.find({'username': u})
    if active_user.count() > 0:
        for s in active_user[0]['shots']:
            tags_count = {'shot': s['id'], 'n_tags': len(s['tags'])}
            shots_tags_count.append(tags_count)
            
    i=i+1
    print i
        
for s in shots_tags_count:
    print s

leggere da file

In [10]:
shots_tags_count = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/tags_per_shot.txt", "r") as ins:
    for line in ins:
        tag = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'shot': tag, 'n_tags': count}
        shots_tags_count.append(elem)

scrivere su file

In [9]:
with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/tags_per_shot.txt", mode='a') as f:
    for s in shots_tags_count:
        f.write(str(s['shot'])+' - '+str(s['n_tags'])+'\n')

NameError: name 'shots_tags_count' is not defined

Stampa gli n shots con piu tag

In [11]:
def n_shots_with_more_tags(n, shot_list):
    max_shot_prev = {'n_tags':1000000}
    for i in range(0,n):
        max = 0
        max_shot = {}
        for s in shot_list:
            if int(s['n_tags']) > max and int(s['n_tags']) < int(max_shot_prev['n_tags']):
                max = int(s['n_tags'])
                max_shot = s
                
        
        max_shot_prev = max_shot
        print max_shot


In [12]:
n_shots_with_more_tags(10, shots_tags_count)

{'n_tags': '63', 'shot': '102858'}
{'n_tags': '38', 'shot': '97672'}
{'n_tags': '30', 'shot': '1104329'}
{'n_tags': '29', 'shot': '896977'}
{'n_tags': '28', 'shot': '740457'}
{'n_tags': '27', 'shot': '1055000'}
{'n_tags': '26', 'shot': '934145'}
{'n_tags': '25', 'shot': '1087264'}
{'n_tags': '24', 'shot': '518010'}
{'n_tags': '23', 'shot': '919606'}


Distribuzione dei tags negli shots

In [13]:
shots_tags_count[0]

{'n_tags': '11', 'shot': '2686725'}

In [14]:
def tags_distribution(shot_list):
    
    values_found = []
    
    tags_freq = []
    
    for i in range(0,len(shot_list)):
        if shot_list[i]['n_tags'] not in values_found:
            values_found.append(shot_list[i]['n_tags'])
            count = 0
            for j in range(i,len(shot_list)):
                if shot_list[j]['n_tags'] == shot_list[i]['n_tags']:
                    count = count + 1
            
            # elem indica quanti shots hanno un certo numero di tags
            elem = {'tag_quantity': shot_list[i]['n_tags'], 'how_many_shots': count}
            tags_freq.append(elem)
            
    return tags_freq

In [15]:
distrib = tags_distribution(shots_tags_count)

In [16]:
distrib

[{'how_many_shots': 28788, 'tag_quantity': '11'},
 {'how_many_shots': 45651, 'tag_quantity': '12'},
 {'how_many_shots': 34675, 'tag_quantity': '10'},
 {'how_many_shots': 39340, 'tag_quantity': '9'},
 {'how_many_shots': 46550, 'tag_quantity': '7'},
 {'how_many_shots': 44010, 'tag_quantity': '8'},
 {'how_many_shots': 47610, 'tag_quantity': '5'},
 {'how_many_shots': 48689, 'tag_quantity': '6'},
 {'how_many_shots': 43302, 'tag_quantity': '4'},
 {'how_many_shots': 33691, 'tag_quantity': '3'},
 {'how_many_shots': 18071, 'tag_quantity': '2'},
 {'how_many_shots': 909, 'tag_quantity': '15'},
 {'how_many_shots': 1131, 'tag_quantity': '14'},
 {'how_many_shots': 18340, 'tag_quantity': '0'},
 {'how_many_shots': 385, 'tag_quantity': '18'},
 {'how_many_shots': 1486, 'tag_quantity': '13'},
 {'how_many_shots': 8227, 'tag_quantity': '1'},
 {'how_many_shots': 387, 'tag_quantity': '19'},
 {'how_many_shots': 608, 'tag_quantity': '16'},
 {'how_many_shots': 261, 'tag_quantity': '20'},
 {'how_many_shots': 301

In [ ]:
def extract_quantity(json):
    try:
        return int(json['tag_quantity'])
    except KeyError:
        return -1

### Media delle visualizzazioni degli shots (per utente)

In [ ]:
shots_views = []

i=0

for u in usernames_no_repeated:
    active_user = db.shots.find({'username': u})
    if active_user.count() > 0:
        for s in active_user[0]['shots']:
            views = {'shot': s['id'], 'views_count': s['views_count']}
            shots_views.append(views)

    i=i+1
    print i
        
for v in shots_views:
    print v

scrivere su file

In [ ]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/shots_views.txt", mode='a') as f:
    for v in shots_views:
        f.write(str(v['shot'])+' - '+str(v['views_count'])+'\n')

leggere su file

In [17]:
shots_visualization_count = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/shots_views.txt", "r") as ins:
    for line in ins:
        shot = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'shot': shot, 'views_count': count}
        shots_visualization_count.append(elem)

In [18]:
shots_visualization_count

[{'n_tags': '1309', 'shot': '2686725'},
 {'n_tags': '1291', 'shot': '2322796'},
 {'n_tags': '184', 'shot': '4006045'},
 {'n_tags': '300', 'shot': '4002904'},
 {'n_tags': '318', 'shot': '3908921'},
 {'n_tags': '326', 'shot': '3882664'},
 {'n_tags': '431', 'shot': '3882540'},
 {'n_tags': '444', 'shot': '3879370'},
 {'n_tags': '396', 'shot': '3869646'},
 {'n_tags': '175', 'shot': '3869174'},
 {'n_tags': '178', 'shot': '3866717'},
 {'n_tags': '196', 'shot': '3860914'},
 {'n_tags': '305', 'shot': '3777583'},
 {'n_tags': '350', 'shot': '3769496'},
 {'n_tags': '295', 'shot': '3765753'},
 {'n_tags': '450', 'shot': '3758274'},
 {'n_tags': '269', 'shot': '3753074'},
 {'n_tags': '426', 'shot': '3745287'},
 {'n_tags': '457', 'shot': '3734596'},
 {'n_tags': '456', 'shot': '3728597'},
 {'n_tags': '489', 'shot': '3725932'},
 {'n_tags': '255', 'shot': '3706031'},
 {'n_tags': '365', 'shot': '3694149'},
 {'n_tags': '274', 'shot': '3690514'},
 {'n_tags': '415', 'shot': '3653318'},
 {'n_tags': '196', 'sho

### Media e valore massimo dei commenti

In [ ]:
shots_comments = []

i=0

for u in usernames_no_repeated:
    active_user = db.shots.find({'username': u})
    if active_user.count() > 0:
        for s in active_user[0]['shots']:
            comments = {'shot': s['id'], 'comments_count': s['comments_count']}
            shots_comments.append(comments)

    i=i+1
    print i
        
for c in shots_comments:
    print c

scrivere su file

In [ ]:
with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/shots_comments.txt", mode='a') as f:
    for v in shots_comments:
        f.write(str(v['shot'])+' - '+str(v['comments_count'])+'\n')

leggere da file

In [24]:
shots_comments = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/shots_comments.txt", "r") as ins:
    for line in ins:
        shot = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'shot': shot, 'comments_count': count}
        shots_comments.append(elem)

media dei commenti per uno shot su tutti gli gli shots di tutti gli utenti attivi

In [28]:
count = 0
for s in shots_comments:
    count = count + float(s['comments_count'])

print 'AVERAGE COMMENTS:'
print float(count)/float(len(shots_comments))
    

AVERAGE COMMENTS:
3.45705196182


valore massimo di commenti

In [29]:
maxc = 0
max_shot = {}
for s in shots_comments:
    if int(s['comments_count']) > maxc:
        maxc = int(s['comments_count'])
        max_shot = s
        
print 'MAX COMMENTS VALUE: ', maxc
print max_shot

MAX COMMENTS VALUE:  457
{'comments_count': '457', 'shot': '1489592'}
